In [113]:
# Package imports
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import pandas as pd
import numpy as np
from decimal import Decimal


In [156]:
# set up training rate alpha
alpha = 0.5

# Use pandas to read the CSV file as a dataframe
df = pd.read_csv("moons400.csv")
# The y values are those labelled 'Class': extract their values
y = df['Class'].values

# The x values are all other columns
del df['Class']    # drop the 'Class' column from the dataframe
X = df.as_matrix() # convert the remaining columns to a numpy array
# Some examples of working with the data, to look at rows/columns
print ("len(X):", len(X))            # outer array: one per sample
print ("len(X[0]):", len(X[0]))      # each inner array is the attributes of one sample
print ("len(X[:,0]):", len(X[:,0]))  # select column 0 from array

# np.shape returns all dimensions of the array
(nsamples, nattribs) = np.shape(X)
print ("X: nsamples =", nsamples, ", nattribs =", nattribs)

len(X): 400
len(X[0]): 2
len(X[:,0]): 400
X: nsamples = 400 , nattribs = 2


In [157]:
datasubset_x = X[0:1]
(dsamples, dattribs) = np.shape(datasubset_x)
print("datasubset: dsamples =", dsamples, ", dattribs =", dattribs)
print(datasubset_x)

datasubset: dsamples = 1 , dattribs = 2
[[ 2.07106946  0.41152931]]


In [158]:
inputs = datasubset_x
# inputs[0,0] = 0.05
# inputs[0,1] = 0.1
print(inputs)

[[ 2.07106946  0.41152931]]


In [159]:
target = y[0:1]
# target = 0.01
print(target)

[1]


In [160]:
# set up weights array and bias variables
Weights1 = np.array([[0.15000,0.20000],[0.25000,0.30000]])
bias_1 = 0.35
print (Weights1)

Weights2 = np.array([[0.40000, 0.450000]])
bias_2 = 0.6
print (Weights2)

[[ 0.15  0.2 ]
 [ 0.25  0.3 ]]
[[ 0.4   0.45]]


In [161]:
# FORWARD PROPAGATION USING FUNCTIONS
# do the above using a function
# convert calculations to function for hidden layer
def get_sigmoid_output(weights, input, bias):
    hidden_layer = weights * inputs
    hidden_layer = np.sum(hidden_layer, axis=1) + bias
    hidden_layer = 1/(1+np.exp(-hidden_layer))
    return hidden_layer;

def get_output(sigmoid_out, weights, bias):
    output = (sigmoid_out*weights)
    output = 1/(1+np.exp(-(np.sum(output)+bias)))
    return output;

# get the error 
# error_o = (0.5*(target - output)**2)
# print(error_o)

#convert to a function
# I think this needs to change to accept a numpy array as well
# we should end up with a data frame we can plot against the number of executions
def get_error(t, o):
    error_o = (0.5*(t - o)**2)
    return error_o;




In [162]:
# FUNCTIONS FOR BACK PROPAGATION OF OUTPUT
# calculate derivative of error at output layer

# deriv_wrt_out = -(target - output)
# change above to function
def deltaErr_wrt_out(targ, outp):
    result = -(targ - outp)
    return result;  

# calculate the derivation of the error output wrt the net
# derivout_wrt_net = output*(1-output)
# change above to function
def deltaOut_wrt_net(outp):
    result = outp*(1-outp)
    return result;   

# change above to function
# calculate derivative of error wrt to output layer weight OLW_Deriv
def deltaErr_ow(deriv_wrt_out, derivout_wrt_net, activation):
    OLW_Deriv = deriv_wrt_out*derivout_wrt_net*activation
   # print(OLW_Deriv)
    return OLW_Deriv;

In [163]:
# FUNCTIONS FOR BACK PROPAGATION OF HIDDEN LAYER
# calculate derivative of error at hidden layer

# deriv_out_wrt_hL =  Weights2 * deriv_wrt_out *derivout_wrt_net
# print (deriv_out_wrt_hL)
# convert above to function
def deltaOut_hL(deriv_wrt_out, derivout_wrt_net, weights):
    deriv_out_wrt_hL = deriv_wrt_out *derivout_wrt_net *  weights
    return deriv_out_wrt_hL;

# deriv_out_wrt_nethL = activation*(1-activation)
# convert above to function
def deltaOut_netHL(activation):
    activation = activation*(1-activation)
    return activation;

# deriv_wrt_wi = deriv_out_wrt_hL*deriv_out_wrt_nethL*Weights1
# convert above to function
def deltaErr_wi(deriv_out_wrt_hL, deriv_out_wrt_nethL, weights):
    deriv_wrt_wi = deriv_out_wrt_hL*deriv_out_wrt_nethL*weights
    return deriv_wrt_wi;

# convert the above to a function
def calc_adjusted_weights(W, deriv):
    W = W - (alpha*deriv)
    return W;

In [164]:
# START OF FORWARD PROPAGATION FUNCTION CALLS
# test get_sigmoid_output 
hidden_layer = get_sigmoid_output(Weights1, inputs, bias_1)
print (hidden_layer)

# test get_output
output = get_output(hidden_layer, Weights2, bias_2)
print (output)

# call error function
error = get_error(target, output)
print(error)

# END OF FORWARD PROPAGATION FUNCTION CALLS


[ 0.67764416  0.72933004]
0.768392436214
[ 0.02682103]


In [165]:
# START OF BACK PROPAGATION FOR OUTPUT LAYER
#test function deltaErr_wrt_out
deltaErr_out = deltaErr_wrt_out(target, output)
print(deltaErr_out)

#test function
deltaOut_net = deltaOut_wrt_net(output)
print(deltaOut_net)

deltaErrtot_ow = deltaErr_ow(deltaErr_out, deltaOut_net, hidden_layer)
print(deltaErrtot_ow)

# END OF BACK PROPAGATION FOR OUTPUT LAYER


[-0.23160756]
0.177965500183
[-0.02793124 -0.03006164]


In [166]:
# START BACK PROPAGATION OF HIDDEN LAYER
# calculate derivative of error at hidden layer

# test function deltaOut_hL
deltaErrOut_hL = deltaOut_hL(deltaErr_out, deltaOut_net, Weights2)
print(deltaErrOut_hL)
    
#test function 
deltaErrOut_netHL = deltaOut_netHL(hidden_layer)
print(deltaErrOut_netHL)

deltaErrH_wi = deltaErr_wi(deltaErrOut_hL, deltaErrOut_netHL, Weights1)
print(deltaErrH_wi)

# same calculation as above but with Weights matrix 
# calculate adjusted weights using function
Weights2 = calc_adjusted_weights(Weights2, deltaErrtot_ow)
Weights1 = calc_adjusted_weights(Weights1, deltaErrH_wi)

# END OF BACK PROPAGATION OF HIDDEN LAYER

[[-0.01648726 -0.01854817]]
[ 0.21844255  0.19740773]
[[-0.00054023 -0.00073231]
 [-0.00090038 -0.00109847]]


Weights2 before: [[ 0.4   0.45]]
Weights2 after: [[ 0.41573791  0.46693828]]
Weights1 before: [[ 0.15  0.2 ]
 [ 0.25  0.3 ]]
Weights1 after: [[ 0.15030439  0.20041262]
 [ 0.25050732  0.30061893]]
